In [7]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import torch

import bvcopula
import utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
path_models = '/home/nina/models'
path_exp = '/home/nina/VRData/Processing/pkls'
beh = 5

In [9]:
animal = 'ST263'
N_max = 23
dayN = 4
day_name = f"Day{dayN}"
exp_pref = f"{animal}_{day_name}"
device = torch.device('cuda:0')

In [10]:
X,_ = utils.load_experimental_data(path_exp, animal, day_name, 0, 1)
S = torch.tensor(X).float().squeeze().to(device)

In [11]:
# points = torch.linspace(0,1,20)
# likelihoods = utils.get_likelihoods(f"{path_models}/{exp_pref}/summary.pkl",n1,n2)
# weights_file = f"{path_models}/{exp_pref}/model_{n1}-{n2}.pth"
# %time model = utils.get_model(weights_file, likelihoods, torch.device('cpu')) 
# %time (ym, y, dy) = model.MI(points)
# plt.plot(np.linspace(0,160,len(y)),y)
# plt.fill_between(np.linspace(0,160,len(y)),y-dy,y+dy, alpha=0.3)
# plt.plot(np.linspace(0,160,len(ym)),ym)
# plt.ylim(-.5,0)

In [14]:
import time

f_mc_size = 5

def get_MI(n1,n2):
    likelihoods = utils.get_likelihoods(f"{path_models}/{exp_pref}/summary.pkl",n1,n2)
    if (len(likelihoods)==1) & (likelihoods[0].name=='Independence'):
        return (0,0)
    else:
        weights_file = f"{path_models}/{exp_pref}/model_{n1}-{n2}.pth"

        model = utils.get_model(weights_file, likelihoods, device) 
        with torch.no_grad():
            Fs = model(S).rsample(torch.Size([f_mc_size])) 
            #[samples_f, copulas, stimuli(positions)]
            MI,sem=model.likelihood.stimMI(S,Fs,s_mc_size=200,r_mc_size=20,sR_mc_size=3000,sem_tol=5*1e-3)
            #print(f"{MI.mean().item():.3}±{MI.std().item():.3} -- {(sem.max()/(MI.max()-MI.min())).item():.3}")

        return (MI.mean(),MI.std())

MI = np.zeros((N_max+beh,N_max+beh,2))

start_time = time.time()
for n1 in range(-5,N_max-1):
    t1 = time.time()
    k=0
    for n2 in range(n1+1,N_max):
        MI[n1+beh,n2+beh,:] = get_MI(n1,n2)
        if MI[n1+beh,n2+beh,0]!=0:
            k+=1
    if k!=0:
        print(n1,' ',(time.time()-t1)/k,' s')
    
total_time = (time.time() - start_time)/60
print(f"Took {total_time:.1f} min overall")

0.00307±0.000952 -- 2.86
0.000168±7.36e-05 -- 37.1
0.00843±0.00211 -- 1.16
2.1647934913635254  s
0.00333±0.000545 -- 5.05
0.0129±0.000864 -- 2.82
0.00679±0.00259 -- 0.745
0.00761±0.00462 -- 0.442
0.0121±0.00404 -- 0.523
0.00682±0.00157 -- 1.58
0.00394±0.00254 -- 0.74
5.6797666890280585  s
0.0054±0.000992 -- 2.82
0.017±0.00224 -- 1.07
0.0223±0.0042 -- 0.516
15.778471310933432  s
0.00932±0.00417 -- 0.519
0.0122±0.00254 -- 0.532
0.00288±0.00126 -- 1.43
0.0157±0.00316 -- 0.704
0.000896±0.000781 -- 2.42
0.0114±0.00309 -- 0.525
4.182692964871724  s
0.021±0.00828 -- 0.297
0.025±0.00128 -- 1.42
0.0117±0.00588 -- 0.48
0.0167±0.00449 -- 0.469
0.00527±0.00117 -- 1.5
0.0163±0.00296 -- 0.55
22.182960669199627  s
0.0113±0.00104 -- 2.72
0.0111±0.00248 -- 0.715
0.00273±0.00089 -- 2.92
0.00433±0.0015 -- 1.65
0.000662±0.000604 -- 3.65
0.00318±0.00251 -- 0.92
0.0013±0.000869 -- 3.05
0.0047±0.000965 -- 3.09
18.631142377853394  s
0.0115±0.00292 -- 0.878
0.00109±0.000773 -- 3.89
0.0183±0.00215 -- 1.39
0.014

In [15]:
with open(f"{path_models}/MI_measures/{exp_pref}_copulaMI.pkl",'wb') as f:
    pkl.dump(MI,f)